In [16]:
#Importando pandas
import pandas as pd


In [17]:
#Carregando dados
loja1 = pd.read_csv('../base-de-dados-challenge-1/loja_1.csv')
loja2 = pd.read_csv('../base-de-dados-challenge-1/loja_2.csv')
loja3 = pd.read_csv('../base-de-dados-challenge-1/loja_3.csv')
loja4 = pd.read_csv('../base-de-dados-challenge-1/loja_4.csv')

In [22]:
#Adicionando coluna com numero da loja
loja1['Loja'] = "Loja 1"
loja2['Loja'] = "Loja 2"
loja3['Loja'] = "Loja 3"
loja4['Loja'] = "Loja 4"
loja4.head(3)

,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,lat,lon,Loja
0,Lava louças,eletrodomesticos,1446.46,74.604083,11/11/2022,Rafael Costa,SP,4,cartao_credito,3,-22.19,-48.79,Loja 4
1,Cama box,moveis,631.70,33.919251,03/01/2020,Thiago Silva,GO,5,cartao_credito,2,-15.98,-49.86,Loja 4
2,Bola de basquete,esporte e lazer,31.54,0.000000,25/12/2022,Larissa Alves,MG,5,boleto,1,-18.10,-44.38,Loja 4


In [23]:
#Concatenar as lojas
lojas = pd.concat([loja1, loja2, loja3, loja4], ignore_index=True)
lojas.head(3)

,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,lat,lon,Loja
0,Assistente virtual,eletronicos,219.08,9.249790,16/01/2021,Pedro Gomes,SP,4,cartao_credito,8,-22.19,-48.79,Loja 1
1,Mesa de jantar,moveis,256.35,11.234305,18/05/2022,Beatriz Moraes,RJ,1,cartao_credito,4,-22.25,-42.66,Loja 1
2,Jogo de tabuleiro,brinquedos,279.51,21.262681,15/03/2021,João Souza,DF,1,cartao_credito,1,-15.83,-47.86,Loja 1


In [29]:
lojas = lojas[["Produto","Loja","lat","lon"]]
lojas.head(2)

,Produto,Loja,lat,lon
0,Assistente virtual,Loja 1,-22.19,-48.79
1,Mesa de jantar,Loja 1,-22.25,-42.66


In [26]:
#Agora vamos obter o nome das cidades, usando os dados de "lat" e "lon"

In [35]:
#Install
#pip install geopy
#pip install geopandas shapely
#pip install geobr


import geobr
import geopandas as gpd
from shapely.geometry import Point


Obter os municipios com geor

In [36]:
municipios = geobr.read_municipality(year=2020)  # Año más reciente disponible
municipios.head(2)

ERROR 1: PROJ: proj_create_from_database: Open of /home/ygor/miniconda3/envs/jupyter/share/proj failed


,code_muni,name_muni,code_state,abbrev_state,name_state,code_region,name_region,geometry
0,1100015.0,Alta Floresta D'oeste,11.0,RO,Rondônia,1.0,Norte,"MULTIPOLYGON (((-62.19465 -11.82746, -62.18945..."
1,1100023.0,Ariquemes,11.0,RO,Rondônia,1.0,Norte,"MULTIPOLYGON (((-62.53648 -9.73222, -62.52765 ..."
